In [1]:
import cv2
import numpy as np
import os

# Set parameters
video_path = "video.mp4"  # Replace with your video file
output_folder = "dataset/extracted_frames/"  # Output folder for extracted frames
inverted_folder = "dataset/inverted_frames/"  # Output folder for inverted frames

# Create output folders if they don't exist
os.makedirs(output_folder, exist_ok=True)
os.makedirs(inverted_folder, exist_ok=True)

# Read video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Cannot open video file.")
    exit()

frame_count = 0
inverted_count = 0  # To track the number of inverted frames

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Get the height and width of the frame
    height, width = frame.shape[:2]

    # Crop the upper 30% of the frame
    crop_start = int(height * 0.3)  # Start cropping from 30% height
    cropped_frame = frame[crop_start:, :]  # Retain only the bottom 70%

    # Save the cropped frame
    cropped_frame_path = os.path.join(output_folder, f"frame_cropped_{frame_count:04d}.jpg")
    cv2.imwrite(cropped_frame_path, cropped_frame)

    # Convert cropped frame to HSV
    hsv = cv2.cvtColor(cropped_frame, cv2.COLOR_BGR2HSV)

    # Define a flexible yellow range
    lower_yellow = np.array([20, 20, 20])  # Adjust these values
    upper_yellow = np.array([40, 255, 255])

    # Create a mask for the yellow region
    mask = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Apply the mask to the cropped frame
    masked_frame = cv2.bitwise_and(cropped_frame, cropped_frame, mask=mask)

    # Save the masked frame
    masked_frame_path = os.path.join(output_folder, f"frame_masked_{frame_count:04d}.jpg")
    cv2.imwrite(masked_frame_path, masked_frame)

    # Save inverted frame based on cropped frame
    if inverted_count < frame_count // 2:  # Ensure inverted frames are not more than half
        inverted_frame = cv2.flip(cropped_frame, 1)  # Flip horizontally
        inverted_frame_path = os.path.join(inverted_folder, f"frame_inverted_{inverted_count:04d}.jpg")
        cv2.imwrite(inverted_frame_path, inverted_frame)
        inverted_count += 1

    frame_count += 1

cap.release()
print(f"Extracted and processed {frame_count} frames to {output_folder}")
print(f"Generated {inverted_count} inverted frames to {inverted_folder}")


Extracted and processed 202 frames to dataset/extracted_frames/
Generated 100 inverted frames to dataset/inverted_frames/
